Compare Tide Forecasts
======================

Compare the daily tidal displacements for a given location with station data

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  
- https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/
- ftp://ftp.esr.org/pub/datasets/tmd/  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  

Finite Element Solution (FES) provided by AVISO  
- https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  

#### Program Dependencies

- `calc_astrol_longitudes.py`: computes the basic astronomical mean longitudes  
- `calc_delta_time.py`: calculates difference between universal and dynamic time  
- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `load_constituent.py`: loads parameters for a given tidal constituent  
- `load_nodal_corrections.py`: load the nodal corrections for tidal constituents  
- `infer_minor_corrections.py`: return corrections for minor constituents  
- `model.py`: retrieves tide model parameters for named tide models  
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  
- `read_netcdf_model.py`: extract tidal harmonic constants from netcdf models  
- `read_GOT_model.py`: extract tidal harmonic constants from GSFC GOT models  
- `read_FES_model.py`: extract tidal harmonic constants from FES tide models  
- `predict_tidal_ts.py`: predict tidal time series at a location using harmonic constants  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  

#### Load modules

In [1]:
from __future__ import print_function

import os
import pandas
import datetime
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display
import urllib.request

import pyTMD.time
import pyTMD.model
import pyTMD.tools
from pyTMD.calc_delta_time import calc_delta_time
from pyTMD.infer_minor_corrections import infer_minor_corrections
from pyTMD.predict_tidal_ts import predict_tidal_ts
from pyTMD.read_tide_model import extract_tidal_constants
from pyTMD.read_netcdf_model import extract_netcdf_constants
from pyTMD.read_GOT_model import extract_GOT_constants
from pyTMD.read_FES_model import extract_FES_constants
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# available model list
model_list = sorted(pyTMD.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.directory.value = os.path.join('~','data.dir','tide_models')
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'TPXO8-atlas'
TMDwidgets.compress.value = False
widgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
    TMDwidgets.atlas,
    TMDwidgets.compress
])

### Get tide gauge data

In [3]:
# Port Kembla
# url = 'http://www.bom.gov.au/ntc/IDO71003/IDO71003_2022.csv'
# LAT,LON = (-34.47375,150.911861)

# # Townsville
# url = 'http://www.bom.gov.au/ntc/IDO71001/IDO71001_2022.csv'
# LAT,LON = (-19.2774,147.0586)

# # Port Stanvac
# url = 'http://www.bom.gov.au/ntc/IDO71009/IDO71009_2010.csv'
# LAT,LON = (-35.1085,138.4671)

# # Spring Bay
# url = 'http://www.bom.gov.au/ntc/IDO71007/IDO71007_2022.csv'
# LAT,LON = (-42.5464,147.9308)

# Stony Point
url = 'http://www.bom.gov.au/ntc/IDO71004/IDO71004_2022.csv'
LAT,LON = (-38.3722,145.2247)

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
request = urllib.request.Request(url, headers=headers)
f_in = urllib.request.urlopen(request, timeout=20)
gauge = pandas.read_csv(f_in)
gauge = gauge[gauge['Sea Level'] != -9999]

In [5]:
# get model parameters
model = pyTMD.model(TMDwidgets.directory.value,
    format=TMDwidgets.atlas.value,
    compressed=TMDwidgets.compress.value
   ).elevation(TMDwidgets.model.value)
# model.format = 'OTIS'

# convert from calendar date to days relative to Jan 1, 1992 (48622 MJD)
utc_time = gauge[' Date & UTC Time'].astype(np.datetime64)
tide_time = pyTMD.time.convert_datetime(utc_time,
    epoch=(1992,1,1,0,0,0))/86400.0
hours = np.arange(len(tide_time))
# delta time (TT - UT1) file
delta_file = pyTMD.utilities.get_data_path(['data','merged_deltat.data'])

# read tidal constants and interpolate to station points
if model.format in ('OTIS','ATLAS'):
    amp,ph,D,c = extract_tidal_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.grid_file, model.model_file,
        model.projection, TYPE=model.type, METHOD='spline',
        EXTRAPOLATE=True, GRID=model.format)
    DELTAT = np.zeros_like(tide_time)
elif (model.format == 'netcdf'):
    amp,ph,D,c = extract_netcdf_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.grid_file, model.model_file,
        TYPE=model.type, METHOD='spline', EXTRAPOLATE=True,
        SCALE=model.scale, GZIP=model.compressed)
    DELTAT = np.zeros_like(tide_time)
elif (model.format == 'GOT'):
    amp,ph,c = extract_GOT_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.model_file, METHOD='spline',
        EXTRAPOLATE=True, SCALE=model.scale,
        GZIP=model.compressed)
    # interpolate delta times from calendar dates to tide time
    DELTAT = calc_delta_time(delta_file, tide_time)
elif (model.format == 'FES'):
    amp,ph = extract_FES_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.model_file, TYPE=model.type,
        VERSION=model.version, METHOD='spline', EXTRAPOLATE=True,
        SCALE=model.scale, GZIP=model.compressed)
    c = model.constituents
    # interpolate delta times from calendar dates to tide time
    DELTAT = calc_delta_time(delta_file, tide_time)

# calculate complex phase in radians for Euler's
cph = -1j*ph*np.pi/180.0
# calculate constituent oscillation
hc = amp*np.exp(cph)

# convert time from MJD to days relative to Jan 1, 1992 (48622 MJD)
# predict tidal elevations at time 1 and infer minor corrections
TIDE = predict_tidal_ts(tide_time, hc, c,
    DELTAT=DELTAT, CORRECTIONS=model.format)
MINOR = infer_minor_corrections(tide_time, hc, c,
    DELTAT=DELTAT, CORRECTIONS=model.format)
TIDE.data[:] += MINOR.data[:]

In [ ]:
%matplotlib widget
# create plot with tidal displacements, high and low tides and dates
fig,ax1 = plt.subplots(num=1)
xmax = np.ceil(hours[-1]).astype('i')
ax1.plot(hours,TIDE.data+np.mean(gauge['Sea Level']),'k')
gauge.plot(y='Sea Level',ax=ax1)
for h in range(24,xmax,24):
    ax1.axvline(h,color='gray',lw=0.5,ls='dashed',dashes=(11,5))
ax1.set_xlim(0,xmax)
ax1.set_ylabel('{0} Tidal Displacement [m]'.format(model.name))
args = (utc_time[0].year,utc_time[0].month,utc_time[0].day)
ax1.set_xlabel('Time from {0:4d}-{1:02d}-{2:02d} UTC [Hours]'.format(*args))
ax1.set_title(u'{0:0.6f}\u00b0N {1:0.6f}\u00b0W'.format(LAT,LON))
fig.subplots_adjust(left=0.10,right=0.98,bottom=0.10,top=0.95)
plt.show()

In [ ]:
fig,ax2 = plt.subplots(num=2,figsize=(6,6))
ax2.plot(gauge['Sea Level'].values,TIDE.data,'.')
ax2.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
plt.close('all')